In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
import wandb
import datetime
import pytz

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('../../../baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../../dataset/'
dataset_idx = 1

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = f'../dataset/train{dataset_idx}.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = f'../dataset/valid{dataset_idx}.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

# wandb setting
kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(kst)
date = now.strftime('%Y_%m_%d_%H_%M_%S_')
project_name = "Mini"
exp_name = date + f"FasterRCNN_SGKF_{dataset_idx}"
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': project_name, 'name' : exp_name},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=0,
         bbox_score_thr=0.3)]

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + exp_name

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3909, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3171  | 1 [Paper]     | 5081  | 2 [Paper pack]  | 718   | 3 [Metal]   | 749   | 4 [Glass]    | 785   |
| 5 [Plastic]       | 2355  | 6 [Styrofoam] | 1011  | 7 [Plastic bag] | 4142  | 8 [Battery] | 128   | 9 [Clothing] | 374   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2023-05-11 13:06:37,023 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2023-05-11 13:06:37,024 - mmcv - INFO - load model from: torchvision://resnet50
2023-05-11 13:06:37,025 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2023-05-11 13:06:37,275 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2023-05-11 13:06:37,302 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-05-11 13:06:37,331 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2023-05-11 13:06:37,340 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [6]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2023-05-11 13:06:41,865 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-11 13:06:41,894 - mmdet - INFO - Start running, host: root@900ed28a7422, work_dir: /opt/ml/level2_objectdetection-cv-06/Hyunjin Jung/code/work_dirs/2023_05_11_22_06_36_FasterRCNN_SGKF_1
2023-05-11 13:06:41,895 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mini123 (cv-06). Use `wandb login --relogin` to force relogin


2023-05-11 13:06:47,777 - mmdet - WARNING - No meta information found in the runner. 
2023-05-11 13:07:02,928 - mmdet - INFO - Epoch [1][50/978]	lr: 1.978e-03, eta: 0:58:59, time: 0.303, data_time: 0.053, memory: 2101, loss_rpn_cls: 0.4195, loss_rpn_bbox: 0.0494, loss_cls: 0.6321, acc: 87.9453, loss_bbox: 0.1017, loss: 1.2027, grad_norm: 7.1397
2023-05-11 13:07:15,426 - mmdet - INFO - Epoch [1][100/978]	lr: 3.976e-03, eta: 0:53:35, time: 0.250, data_time: 0.008, memory: 2104, loss_rpn_cls: 0.1661, loss_rpn_bbox: 0.0518, loss_cls: 0.3190, acc: 93.9785, loss_bbox: 0.2289, loss: 0.7658, grad_norm: 2.3735
2023-05-11 13:07:27,246 - mmdet - INFO - Epoch [1][150/978]	lr: 5.974e-03, eta: 0:50:48, time: 0.237, data_time: 0.007, memory: 2104, loss_rpn_cls: 0.1522, loss_rpn_bbox: 0.0565, loss_cls: 0.3301, acc: 93.8838, loss_bbox: 0.2301, loss: 0.7690, grad_norm: 2.5736
2023-05-11 13:07:38,817 - mmdet - INFO - Epoch [1][200/978]	lr: 7.972e-03, eta: 0:49:03, time: 0.231, data_time: 0.008, memory: 2

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 974/974, 23.1 task/s, elapsed: 42s, ETA:     0s

...

2023-05-11 14:05:12,072 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.93s).
Accumulating evaluation results...


2023-05-11 14:05:17,391 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.362
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.254
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.432
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.432
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.432
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.044
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.232
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.04s).
Loading and preparing results...
DONE (t=0.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.67s).
Accumulating evaluation results...
DONE (t=1.29s).


learning_rate,▃▅█████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▆▆▂▇▅▅▄▄▄▄▃▆▅▄▁▅▅▅▅▄▅▅▅▃▆▆▄▅▄▅▅▆▅▆█▆▇▇█▅
train/grad_norm,███▂▃▁▅▃▃▂▃▂▄▄▇▃▄▄▄▄▅▄▄▆▄▄▇▄▅▃▅▅▆▅▃▆▄▃▃▇
train/loss,▅▅█▃▄▃▅▄▄▄▄▃▃▄▆▃▃▃▃▄▃▃▃▄▃▂▃▃▄▂▃▂▃▂▁▂▂▁▁▃
train/loss_bbox,▅▅█▂▄▃▄▄▄▅▅▂▄▅▇▄▃▄▃▆▃▄▄▄▃▂▄▄▆▃▄▃▄▃▁▃▃▂▁▅
train/loss_cls,▅▅█▃▄▄▅▅▄▄▅▃▄▄▆▃▃▃▃▄▄▃▄▄▂▃▄▃▃▃▃▂▃▂▁▃▂▁▁▃
train/loss_rpn_bbox,▅▄█▂▃▃▄▃▂▃▄▃▃▄▆▃▄▃▂▆▂▃▂▃▃▃▂▅▄▂▃▂▃▂▁▃▂▂▂▄
train/loss_rpn_cls,█▆█▅▄▃▄▃▃▄▄▃▂▄▄▂▄▂▂▄▂▃▂▃▃▂▂▂▂▁▂▁▂▁▁▂▁▁▁▂
val/bbox_mAP,▁▂▂▄▃▄▅▆▇███
val/bbox_mAP_50,▁▂▂▄▃▄▅▆▇███


In [ ]:
wandb.finish()